In [17]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Dict, NamedTuple, Union
import logging
from matplotlib import pyplot as plt
import time
from itertools import product
import gc
import pathlib
from glob import glob
from dateutil.relativedelta import relativedelta
from tqdm import tqdm 
import os
import json

In [18]:
class Parameters(NamedTuple):
    split: int
    repeat: int
    epochs: int
    steps_per_epoch: int
    block_layers: int
    hidden_units: int
    num_blocks: int
    block_sharing: bool
    horizon: int
    history_lookback: int
    init_learning_rate: float
    decay_steps: int
    decay_rate: float
    loss: str
    pinball_tau: float
    nmse_weight: float
    pnmse_weights: str
    batch_size: int
    weight_decay: float
    ts_sampling: str



hyperparams_dict = {
    "split": 0, # 0 is test split, 1 is validation split
    "repeat": list(range(0, 1024)), 
    "epochs": 20, 
    "steps_per_epoch": [100], 
    "block_layers": [3],
    "hidden_units": 512,
    "num_blocks": 6,
    "block_sharing": True, # [True, False]
    "horizon": 12,
    "history_lookback": 1,
    "init_learning_rate": 1e-3,
    "decay_steps": 3,
    "decay_rate": 0.5,
    "loss": ["pnmse"],  # ["pnmse", "pinball", "mape", "smape", "pmape", "rmse"]
    "pnmse_weights": ["mape"], # ["mape", "smape"]
    "pinball_tau": [0.35], # This is selected to minimize the bias on the validation set
    "nmse_weight": [0.35],
    "batch_size": 256,
    "weight_decay": 0,
    "ts_sampling": ["ts_weight"], # ["uniform", "ts_weight"]
}

HORIZON = 12

hyperparams = Parameters(**hyperparams_dict)


In [19]:
class ElectricityLoader:
    def __init__(self, path="data/", split='train'):
        self.path = path
        self.split = split
        ts_raw = pd.read_csv(os.path.join(path,f'Electricity-{split}.csv')).iloc[:,1:].values.astype(np.float32)
        self.ts_raw = []
        
        for ts in ts_raw:
            self.ts_raw.append(ts[~np.isnan(ts)])

        self.ts_weight = np.zeros((len(self.ts_raw),), dtype=np.float32)
        for i,ts in enumerate(self.ts_raw):
            self.ts_weight[i] = len(ts)
        self.ts_weight = self.ts_weight / self.ts_weight.sum()

    def get_val_split(self, split_horizon=1):
        train_dataset = ElectricityLoader(path=self.path, split='train')
        test_dataset = ElectricityLoader(path=self.path, split='train')
        ts_raw = []
        for ts in train_dataset.ts_raw:
            ts_raw.append(ts[:-split_horizon*12])
        train_dataset.ts_raw = ts_raw
        ts_raw = []
        for ts in test_dataset.ts_raw:
            ts_raw.append(ts[-split_horizon*12:])
        test_dataset.ts_raw = ts_raw
        return train_dataset, test_dataset

    def get_batch(self, batch_size=64, win_len=14*2, horizon=12, ts_sampling='uniform'):
        target_ts = np.zeros((batch_size, horizon), dtype=np.float32)
        history_ts = np.zeros((batch_size, win_len), dtype=np.float32)

        if ts_sampling == "uniform":
            ts_idxs = np.random.choice(np.arange(len(self.ts_raw)), size=batch_size, replace=True)
        elif ts_sampling == "ts_weight":
            ts_idxs = np.random.choice(np.arange(len(self.ts_raw)), size=batch_size, replace=True, p=self.ts_weight)
            
        for i, ts_id in enumerate(ts_idxs):
            ts = self.ts_raw[ts_id]
            sampling_point = np.random.choice(np.arange(win_len, len(ts)-horizon+1), size=1, replace=False)[0]
            history_ts[i,:] = ts[sampling_point-win_len:sampling_point]
            target_ts[i,:] = ts[sampling_point:sampling_point+horizon]

        batch = {"history": history_ts, "target": target_ts}
        return batch


    def get_sequential_batch(self, win_len=14*2):
        history_ts = np.zeros((len(self.ts_raw), win_len), dtype=np.float32)
        for i, ts in enumerate(self.ts_raw):
            history_ts[i,:] = ts[-win_len:]
        return {"history": history_ts}

In [20]:
class NBeatsBlock(tf.keras.layers.Layer):
        def __init__(self, hyperparams: Parameters, input_size:int, output_size:int, **kw):
            super(NBeatsBlock, self).__init__(**kw)
            self.hyperparams = hyperparams
            self.input_size = input_size
            self.output_size = output_size
            self.fc_layers = []
            for i in range(hyperparams.block_layers):
                self.fc_layers.append(
                    tf.keras.layers.Dense(hyperparams.hidden_units,
                                        activation=tf.nn.relu,
                                        kernel_regularizer=tf.keras.regularizers.l2(hyperparams.weight_decay),
                                        name=f"fc_{i}")
                )
            self.forecast = tf.keras.layers.Dense(output_size, activation=None, name="forecast")
            self.backcast = tf.keras.layers.Dense(self.input_size, activation=None, name="backcast")

        def call(self, inputs, training=False):
            epsilon = 1e-6 

            h = self.fc_layers[0](inputs)
            for i in range(1, self.hyperparams.block_layers):
                h = self.fc_layers[i](h)
            backcast = self.backcast(h)



            mu = tf.reduce_mean(inputs, axis=-1, keepdims=True)
            std = tf.math.reduce_std(inputs, axis=-1, keepdims=True)
            std = tf.maximum(std, epsilon)  # Prevent std from being too small

            backcast = backcast * std + mu
            backcast = tf.keras.activations.relu(inputs - backcast)

            forecast = self.forecast(h)
            forecast = forecast * std + mu
            return backcast, forecast

In [21]:
class NBeats:
    def __init__(self, hyperparams: Parameters, name: str='NBeats', logdir: str='logs', num_nodes: int = 100):
        super(NBeats, self).__init__()
        self.hyperparams = hyperparams
        self.name = name
        self.logdir = logdir
        self.num_nodes = num_nodes
        self.input_size = self.hyperparams.history_lookback * self.hyperparams.horizon

        self.nbeats_layers = []
        self.nbeats_layers.append(NBeatsBlock(hyperparams=hyperparams,
                                              input_size=self.input_size,
                                              output_size=hyperparams.horizon,
                                              name=f"nbeats_{0}")
                                  )
        for i in range(1, hyperparams.num_blocks):
            if self.hyperparams.block_sharing:
                self.nbeats_layers.append(self.nbeats_layers[0])
            else:
                self.nbeats_layers.append(NBeatsBlock(hyperparams=hyperparams,
                                                      input_size=self.input_size,
                                                      output_size=hyperparams.horizon,
                                                      name=f"nbeats_{i}")
                                          )

        inputs, outputs = self.get_model()
        model = tf.keras.Model(inputs=inputs, outputs=outputs, name=name)
        self.inputs = inputs
        self.outputs = outputs
        self.model = model

    def get_model(self):
        history_in = tf.keras.layers.Input(shape=(self.hyperparams.history_lookback * self.hyperparams.horizon,), name='history')

        level = tf.keras.layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(history_in)
        history_delevel = tf.keras.layers.Lambda(lambda x: tf.math.divide_no_nan(x[0], x[1]))([history_in, level])

        backcast, forecast = self.nbeats_layers[0](inputs=history_delevel)
        for nb in self.nbeats_layers[1:]:
            backcast, forecast_layer = nb(inputs=backcast)
            forecast = tf.keras.layers.Add()([forecast, forecast_layer])

        forecast = tf.keras.layers.Multiply()([forecast, level])

        inputs = {'history': history_in}
        outputs = {'target': forecast}
        return inputs, outputs

    def forecast(self, train_dataset):
        input_data = train_dataset.get_sequential_batch(win_len=self.hyperparams.horizon * self.hyperparams.history_lookback)
        return self.model.predict({"history": input_data["history"]})['target']


In [22]:
def smape(labels, preds):
    weights = tf.stop_gradient(
        tf.math.divide_no_nan(2.0, (tf.abs(preds) + tf.abs(labels))))
    return tf.reduce_mean(tf.abs(preds - labels) * weights)

def mape(labels, preds):
    weights = tf.math.divide_no_nan(1.0, tf.abs(labels))
    return tf.reduce_mean(tf.abs(preds - labels) * weights)

def get_pmape_loss(tau):
    def pmape_loss(labels, preds):
        weights = tf.math.divide_no_nan(1.0, tf.abs(labels))
        pinball = tf.where(labels > preds,
                           x=tau*(labels - preds), 
                           y=(1-tau)*(preds-labels))
        return tf.reduce_mean(pinball * weights)
    return pmape_loss

def get_pinball_loss(tau):
    def pinball_loss(labels, preds):
        pinball = tf.where(labels > preds,
                           x=tau*(labels - preds), 
                           y=(1-tau)*(preds-labels))
        return tf.reduce_mean(pinball)
    return pinball_loss

def mae(labels, preds):
    return tf.reduce_mean(tf.abs(preds - labels))

def rmse(labels, preds):
    rmse_values = tf.sqrt(tf.reduce_mean(tf.square(preds - labels), axis=1))
    return tf.reduce_mean(rmse_values)





def get_pnmse(tau, nmse_weight, weights):
    def pnmse_loss(labels, preds):    


        pinball = tf.where(labels > preds,
                           x=tau * (labels - preds), 
                           y=(1 - tau) * (preds - labels))
        pinball_loss = tf.reduce_mean(pinball * weights(labels, preds))
        
        variance = tf.math.reduce_variance(labels, axis=0)
        nmse = tf.reduce_mean(tf.square(preds - labels) / (variance + 1e-6))
        
        return pinball_loss + nmse_weight * nmse
    return pnmse_loss


def smape_weights(labels, preds):
    return tf.stop_gradient( tf.math.divide_no_nan(2.0, (tf.abs(preds) + tf.abs(labels))))


def mape_weights(labels, preds):
    return tf.math.divide_no_nan(1.0, tf.abs(labels))


class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_dataset, test_dataset, hyperparams):
        super().__init__()
        self.input_data = train_dataset.get_sequential_batch(win_len=hyperparams.horizon*hyperparams.history_lookback)
        self.target = np.array(test_dataset.ts_raw)

    def on_train_begin(self, logs={}):
        pass    

    def on_epoch_end(self, epoch, logs={}):
        prediction_test = self.model.predict({"history": self.input_data["history"]})
        logs['smape_test'] = smape(preds=prediction_test['target'], labels=self.target)
        logs['mape_test'] = mape(preds=prediction_test['target'], labels=self.target)

class Trainer:
    def __init__(self, hyperparams: Parameters, logdir: str):
        inp = dict(hyperparams._asdict())
        values = [v if isinstance(v, list) else [v] for v in inp.values()]
        self.hyperparams = [Parameters(**dict(zip(inp.keys(), v))) for v in product(*values)]
        inp_lists = {k: v  for k, v in inp.items() if isinstance(v, list)}
        values = [v for v in inp_lists.values()]
        variable_values = [dict(zip(inp_lists.keys(), v)) for v in product(*values)]
        folder_names = []
        for d in variable_values: 
            folder_names.append(
                ';'.join(['%s=%s' % (key, value) for (key, value) in d.items()])
            )
        self.history = []
        self.forecasts = []
        self.models = []
        self.logdir = logdir
        self.folder_names = folder_names
        for i, h in enumerate(self.hyperparams): 
            self.models.append(NBeats(hyperparams=h, name=f"nbeats_model_{i}", 
                                      logdir=os.path.join(self.logdir, folder_names[i])))
            
    def generator(self, ds, hyperparams: Parameters):
        while True:
            batch = ds.get_batch(batch_size=hyperparams.batch_size, 
                                 win_len=hyperparams.horizon*hyperparams.history_lookback, 
                                 horizon=hyperparams.horizon,
                                 ts_sampling=hyperparams.ts_sampling)

            yield  {"history": batch["history"]}, {"target": batch["target"]}

    def save_forecast(self, forecast: np.ndarray, filename: str = "forecast.npy"):
        np.save(f"{WORKING_PATH}/{filename}", forecast)
                            
    def fit(self, train_dataset, test_dataset, verbose=1):
        for i, hyperparams in enumerate(self.hyperparams):
            if verbose > 0:
                print(f"Fitting model {i+1} out of {len(self.hyperparams)}, {self.folder_names[i]}")

            path = f"results/{MODEL_VERSION}_split{self.models[i].hyperparams.split}/"
            pathlib.Path(f"{WORKING_PATH}/{path}").mkdir(parents=True, exist_ok=True)
            filename = os.path.join(path, self.folder_names[i]+'.npy')
            if os.path.exists(f"{WORKING_PATH}/{filename}"):
                continue
            
            boundary_step = hyperparams.epochs // 10
            boundary_start = hyperparams.epochs - boundary_step * hyperparams.decay_steps - 1

            boundaries = list(range(boundary_start, hyperparams.epochs, boundary_step))
            values = [float(hyperparams.init_learning_rate * hyperparams.decay_rate ** i) for i in range(len(boundaries) + 1)]
            
            def scheduler(epoch, lr):
                index = np.searchsorted(boundaries, epoch, side='right')
                return float(values[index])

            lr = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

            metrics = MetricsCallback(train_dataset=train_dataset, test_dataset=test_dataset, hyperparams=hyperparams)
            # tb = tf.keras.callbacks.TensorBoard(log_dir=self.models[i].logdir, embeddings_freq=10)

            if hyperparams.loss == 'smape':
                loss = smape
            elif hyperparams.loss == 'mape':
                loss = mape
            elif hyperparams.loss == 'mae':
                loss = mae
            elif hyperparams.loss == 'rmse':
                loss = rmse
            elif hyperparams.loss == 'pmape':
                loss = get_pmape_loss(hyperparams.pinball_tau)
            elif hyperparams.loss == 'pinball':
                loss = get_pinball_loss(hyperparams.pinball_tau)
            elif hyperparams.loss == 'pnmse':
                if hyperparams.pnmse_weights == 'smape':
                    loss = get_pnmse(hyperparams.pinball_tau, hyperparams.nmse_weight, smape_weights)
                elif hyperparams.pnmse_weights == 'mape':
                    loss = get_pnmse(hyperparams.pinball_tau, hyperparams.nmse_weight, mape_weights)
                else:
                    raise ValueError(f"Unknown pnmse weights function: {hyperparams.pnmse_weights}")
            else:
                raise ValueError(f"Unknown loss function: {hyperparams.loss}")
            
            self.models[i].model.compile(optimizer=tf.keras.optimizers.Adam(),
                                        loss=loss)

            fit_output = self.models[i].model.fit(self.generator(ds=train_dataset, hyperparams=hyperparams),
                                                callbacks=[lr, metrics],  # tb
                                                epochs=hyperparams.epochs, 
                                                steps_per_epoch=hyperparams.steps_per_epoch, 
                                                verbose=verbose)
            self.history.append(fit_output.history)

            model_forecast = self.models[i].forecast(train_dataset)
            self.save_forecast(forecast=model_forecast, filename=filename)


In [23]:
def get_forecasts(path, filt):
    a = []
    for f in tqdm(glob(os.path.join(WORKING_PATH, path, filt))):
        df = np.load(f)
        a.append(df)
    return a


def get_ensemble(forecasts):
    return np.mean(np.stack(forecasts, axis=-1), axis=-1)


def get_metrics(preds, labels):
    metrics = {}
    metrics["smape"] = 100*smape(preds=preds, labels=labels).numpy()
    metrics["mape"] = 100*mape(preds=preds, labels=labels).numpy()
    pe = 100*(preds-labels)/labels
    metrics["pe_mean"] = np.mean(pe)
    metrics["pe_median"] = np.median(pe)
    return metrics


def get_stats(samples=10, ensemble_size=64, test_dataset=None, config_filt=None, path=None):
    files = glob(os.path.join(WORKING_PATH, path, filt))
    all_repeats = set([int(f.split(os.sep)[-1].split(";")[0].split("=")[-1]) for f in files])
    preds = np.array(get_forecasts(path=path, filt=config_filt+".npy"))

    metric_samples = []
    ensemble_samples = []
    for s in range(samples):
        ensemble_repeats = np.random.choice(list(all_repeats), size=ensemble_size, replace=False)
        ensemble = preds[ensemble_repeats].mean(axis=0)
        metric_samples.append(get_metrics(preds=ensemble, labels=test_dataset.ts_raw))
        ensemble_samples.append(ensemble)

    return pd.DataFrame(metric_samples), ensemble_samples


def save_ensemble_files(path, ensembles):
    pathlib.Path(f"{WORKING_PATH}/{path}").mkdir(parents=True, exist_ok=True)
    for i, e in enumerate(ensembles):
        filename = os.path.join(path, f"{i}.csv")
        df = pd.DataFrame(data=e, columns=[f"V{i+1}" for i in range(12)], index=[f"P{i+1}" for i in range(35)])
        df.to_csv(filename)

In [25]:
if __name__ == "__main__":
    PROJECT_ROOT = os.getcwd()
    DATASETS_PATH = os.path.join(PROJECT_ROOT, "data")
    WORKING_PATH = PROJECT_ROOT
    LOGDIR = os.path.join(WORKING_PATH, "logs")
    MODEL_VERSION = 'Enhanced_NBEATS'

    os.makedirs(DATASETS_PATH, exist_ok=True)
    os.makedirs(LOGDIR, exist_ok=True)

    print(f"Project Root: {PROJECT_ROOT}")
    print(f"Datasets Path: {DATASETS_PATH}")
    print(f"Logs Directory: {LOGDIR}")
    print(f"Model Version: {MODEL_VERSION}")

    train_dataset = ElectricityLoader(path='data', split='train')
    test_dataset = ElectricityLoader(path='data', split='test')

    trainer = Trainer(hyperparams=hyperparams, logdir=LOGDIR)
    trainer.fit(train_dataset=train_dataset, test_dataset=test_dataset)

    split = 0
    path = f"results/{MODEL_VERSION}_split{split}/"
    filt = "*repeat=*;steps_per_epoch=100*block_layers=3*loss=pnmse*pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight*"

    forecasts = get_forecasts(path, filt)
    a = get_ensemble(forecasts)
    print("SMAPE:", 100 * smape(preds=a, labels=test_dataset.ts_raw).numpy())
    print("MAPE:", 100 * mape(preds=a, labels=test_dataset.ts_raw).numpy())
    print("RMSE:", rmse(preds=a, labels=test_dataset.ts_raw).numpy())

    ensemble_size = 64

    metrics_pnmse, ensembles_pnmse = get_stats(samples=100, ensemble_size=ensemble_size, 
                                               path=f"results/{MODEL_VERSION}_split{split}/", 
                                               test_dataset=test_dataset,
                                               config_filt=filt)

    print(f"PNMSE(MAPE) loss bootstrap stats, ensemble of {ensemble_size} sampled without replacement from 1024 models")
    print(metrics_pnmse.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]))

    path = f"results/{MODEL_VERSION}_split{hyperparams.split}/ensemble_{filt}"
    save_ensemble_files(path=path, ensembles=ensembles_pnmse)


Project Root: /home/aeris/Final
Datasets Path: /home/aeris/Final/data
Logs Directory: /home/aeris/Final/logs
Model Version: Enhanced_NBEATS
Fitting model 1 out of 1024, repeat=0;steps_per_epoch=100;block_layers=3;loss=pnmse;pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight
Fitting model 2 out of 1024, repeat=1;steps_per_epoch=100;block_layers=3;loss=pnmse;pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight
Fitting model 3 out of 1024, repeat=2;steps_per_epoch=100;block_layers=3;loss=pnmse;pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight
Fitting model 4 out of 1024, repeat=3;steps_per_epoch=100;block_layers=3;loss=pnmse;pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight
Fitting model 5 out of 1024, repeat=4;steps_per_epoch=100;block_layers=3;loss=pnmse;pinball_tau=0.35;nmse_weight=0.35;pnmse_weights=mape;ts_sampling=ts_weight
Fitting model 6 out of 1024, repeat=5;steps_per_epoch=100;block_l

100%|█████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:00<00:00, 8069.91it/s]


SMAPE: 3.4977909177541733
MAPE: 3.421401232481003
RMSE: 303.99985


100%|████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:00<00:00, 43279.03it/s]


PNMSE(MAPE) loss bootstrap stats, ensemble of 64 sampled without replacement from 1024 models
            smape        mape     pe_mean   pe_median
count  100.000000  100.000000  100.000000  100.000000
mean     3.524106    3.447664   -0.575070   -0.076714
std      0.043439    0.032293    0.117054    0.052203
min      3.448971    3.386695   -0.876058   -0.283696
5%       3.462191    3.396183   -0.763341   -0.162105
25%      3.491903    3.427433   -0.672586   -0.099194
50%      3.521208    3.446854   -0.570352   -0.072628
75%      3.548922    3.467173   -0.478404   -0.047017
95%      3.595310    3.496977   -0.402131    0.007844
max      3.725731    3.568993   -0.306153    0.036943
